In [210]:
%matplotlib inline
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import log_loss

# Load Data

In [185]:
data_dir = '/Users/nora/Box/dssg/DrivenData/Heart/data/'

In [218]:
test = pd.read_csv(data_dir + 'test_values.csv')
train = pd.read_csv(data_dir + 'train_values.csv')
labels = pd.read_csv(data_dir + 'train_labels.csv')

In [187]:
npatients = len(train)

# Prepare Data

In [188]:
def convert_columns(df, cols=[]):
    for col in cols:
        df[col] = df[col].astype(str)
    return df
        
def standardize(df, numeric_only=True):
    numeric = df.select_dtypes(include=['int64', 'float64'])
    
    # subtracy mean and divide by std
    df[numeric.columns] = (numeric - numeric.mean()) / numeric.std()
    
    return df

def pre_process_data(df, enforce_cols=None):
    print("Input shape:\t{}".format(df.shape))
        

    df = standardize(df)
    print("After standardization {}".format(df.shape))
        
    # create dummy variables for categoricals
    df = pd.get_dummies(df)
    print("After converting categoricals:\t{}".format(df.shape))
    

    # match test set and training set columns
    if enforce_cols is not None:
        to_drop = np.setdiff1d(df.columns, enforce_cols)
        to_add = np.setdiff1d(enforce_cols, df.columns)

        df.drop(to_drop, axis=1, inplace=True)
        df = df.assign(**{c: 0 for c in to_add})
    
    df.fillna(0, inplace=True)
    
    return df

## Prepare Train

In [219]:
patient_id = train['patient_id'].as_matrix()
train = train.drop('patient_id', axis=1)
train = convert_columns(train, cols=['chest_pain_type', 'resting_ekg_results'])
train = pre_process_data(train)

Input shape:	(180, 13)
After standardization (180, 13)
After converting categoricals:	(180, 20)


In [220]:
train.head()

,slope_of_peak_exercise_st_segment,resting_blood_pressure,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina,thal_fixed_defect,thal_normal,thal_reversible_defect,chest_pain_type_1,chest_pain_type_2,chest_pain_type_3,chest_pain_type_4,resting_ekg_results_0,resting_ekg_results_1,resting_ekg_results_2
0,-0.888762,-0.194652,-0.716404,-0.437019,1.115158,-0.900694,0.670152,-1.051032,0.929891,-0.678852,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.727169,-1.252825,-0.716404,-0.437019,-0.667915,0.526148,-1.483908,-0.086892,0.386007,-0.678852,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,-0.888762,-0.371014,2.378462,-0.437019,1.039283,-0.900694,0.670152,2.377024,0.567302,1.464891,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,-0.888762,1.216246,-0.716404,-0.437019,-0.497195,-0.900694,0.670152,-1.586666,1.428452,-0.678852,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,2.343100,2.744719,-0.716404,-0.437019,0.394342,2.844768,0.670152,0.448742,-0.203201,-0.678852,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


## Prepare Labels

In [221]:
labels = np.ravel(labels.drop('patient_id', axis=1))

## Prepare Test

In [222]:
test_patient_id = test['patient_id'].as_matrix()
test = test.drop('patient_id', axis=1)
test = convert_columns(test, cols=['chest_pain_type', 'resting_ekg_results'])
test = pre_process_data(test)

Input shape:	(90, 13)
After standardization (90, 13)
After converting categoricals:	(90, 20)


## Split Data

In [223]:
cutoff = int(npatients*0.75)
train1 = train.iloc[:cutoff]
train2 = train.iloc[cutoff:]
labels1 = labels[:cutoff]
labels2 = labels[cutoff:]
print train1.shape, train2.shape

(135, 20) (45, 20)


# Prepare Submission

In [256]:
def prepare_submission(model, test, fname='submission.sv'):
    probs = model_a.predict_proba(test)
    sub = pd.DataFrame(np.vstack([test_patient_id, probs[:,1]]).T)
    sub.to_csv(fname, sep=',', header=['patient_id','heart_disease_present'], index=False)

# Random Forest

In [227]:
# random forests are often a good model to try first, especially when we have numeric and categorical variables in our feature space.
def train_model(features, labels, **kwargs):
    
    # instantiate model
    # model = RandomForestClassifier(n_estimators=50, random_state=0)
    model = ExtraTreesClassifier(n_estimators=50, random_state=0)
    
    # train model
    model.fit(features, labels)
    
    # get a (not-very-useful) sense of performance
    accuracy = model.score(features, labels)
    # print(f"In-sample accuracy: {accuracy:0.2%}")
    print("In-sample accuracy: %.2f percent" %(accuracy*100))
    
    return model

In [238]:
model_a = train_model(train1, labels1)

In-sample accuracy: 100.00 percent


In [239]:
preds = model_a.predict(train2)
probs = model_a.predict_proba(train2)

In [242]:
print 'Accuracy: %.2f' %(len(np.where(preds == labels2))/len(preds))
print 'Log loss: %.2f' %(log_loss(labels2, probs))

Accuracy: 0.02
Log loss: 0.70


In [243]:
model_b = train_model(train, labels)

In-sample accuracy: 100.00 percent


In [244]:
probs = model_b.predict_proba(test)
preds = model_b.predict(test)

In [248]:
sub = pd.DataFrame(np.vstack([test_patient_id, probs[:,1]]).T)

In [255]:
sub.to_csv('submission1.csv', sep=',', header=['patient_id','heart_disease_present'], index=False)

In [257]:
prepare_submission(model_b, test, 'submission.csv')